In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch', 'port': 9200}])

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [148]:
import time
def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

In [2]:
es.indices.get_alias("*")

{u'.apm-agent-configuration': {u'aliases': {}},
 u'.kibana_1': {u'aliases': {u'.kibana': {}}},
 u'.kibana_task_manager_1': {u'aliases': {u'.kibana_task_manager': {}}},
 u'montevideo': {u'aliases': {}}}

In [57]:
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)

In [109]:
text = 'Siento mucha tristeza al ver esto en Montevideo. Una persona compartió esta imagen que ocurrió hace minutos en la esquina de Salto y (Bernabé) Rivera. La cantidad de personas en situación de calle creció en la capital. Hay planes para mejorar esta realidad, pero duele verla.'

In [94]:
text = 'Mucha basura suelta por Varela frente al Policial'

In [161]:
text = 'En Montevideo podés ir a Porongos esquina Blandengues Upside-down face'

In [182]:
text = '18 de Julio esquina Aquiles Lanza, Montevideo, Uruguay'

In [187]:
text = 'Roban estación de servicio  esquina Montevideo shopping. Hace una semana robaron una joyería a 50 metros de esa estación de servicio. Lugar de mucho tránsito de personas y vehículos. Lleno de camaras de video vigilancia y PADO Man police officerDown pointing backhand index'

In [195]:
text = 'En esta foto estas en que lugar de Europa, amorosa? Ésta estuvo en el cerro de montevideo, calle Venezuela esquina Bogotá....Jaaaa....sos una genia! Smiling face with open mouth and smiling eyes'

In [202]:
text = 'Construction sign👷🏻‍| #MontevideoMejora Round pushpin Av. Italia de Gallinal a Bolivia, luce más amplia y linda Bus Un carril más para el bus Splashing sweat symbol Mejor captación de aguas pluviales 🅿 Refugios Bicycle Bicisenda Electric light bulb 40 nuevas columnas de alumbrado y 80 luminarias LED Más info: Mobile phone with rightwards arrow at left http://bit.ly/3bbb3jA'

In [203]:
results = ic.analyze(index="montevideo", body=
{
  "text": text, 
    "analyzer":"calle_analyzer"
})
for result in results['tokens']:
    print(result['token'])

construction
sign
👷🏻‍
montevideomejora
round
pushpin
av
italia
gallinal
bolivia
luce
mas
amplia
linda
bus
carril
mas
bus
splashing
sweat
symbol
mejor
captacion
aguas
pluviales
🅿
refugios
bicycle
bicisenda
electric
light
bulb
40
nuevas
columnas
alumbrado
80
luminarias
led
mas
info
mobile
phone
with
rightwards
arrow
at
left
http
bit.ly
3bbb3ja


In [211]:
#@timing
def search(text):
    return es.search(index="montevideo", body=
        {
            "from" : 0, "size" : 200,
              "query": {
                "simple_query_string" : {
                    "query": text,
                    "analyzer": "calle_analyzer"
                }
              }
        })['hits']['hits']

In [212]:
#@timing
def search_geo(id):
    return es.search(index="montevideo", body=
    {
        "from" : 0, "size" : 100,
        "query" : {
            "bool": {
                "filter": {
                    "geo_shape": {
                        "geometry": {
                            "indexed_shape": {
                                "index": "montevideo",
                                "id": id,
                                "path": "geometry"
                            }
                        }
                    }
                }
            }
        }
    })['hits']['hits']

In [213]:
@timing
def complete_search(text):
    matchs = []
    state = {}
    results = search(text)
    filtered_results = []
    
    for result in results:
        if (result['_source']['type'] != 'limites_barrios'):
            state[result['_id']] = {"_source":result['_source'],"_score":result['_score']}
            filtered_results.append(result)

    for result in filtered_results:
        nombre = result['_source']['nombre'].encode('ascii', 'ignore').decode('ascii')
        print('{}\t\t{}\t\t{} {}'.format(result['_id'],nombre,result['_source']['geometry']['type'],result['_score']))
        for geo_result in search_geo(result["_id"]):
            geo_nombre = 'NO_NAME'
            if geo_result['_source'].get('nombre',None):
                geo_nombre = geo_result['_source']['nombre'].encode('ascii', 'ignore').decode('ascii')
            elif geo_result['_source'].get('aliases',None):
                geo_nombre = geo_result['_source']['aliases'].encode('ascii', 'ignore').decode('ascii')
            if geo_result['_id'] != result['_id'] and geo_nombre != nombre:
                print('\t{}\t{}\t\t{}'.format(geo_result['_id'], geo_nombre, geo_result['_source']['geometry']['type']))
                if geo_result['_id'] in state.keys():
                    print ('MATCH: {}\t{}\t/\t{}\t{}'.format(nombre,result['_score'],geo_nombre,state[geo_result['_id']]['_score']))
                    matchs.append([result["_id"],geo_result['_id']])
    return (matchs,state)

In [214]:
(matchs,state) = complete_search(text)

geonames_uy_montevideo-8521749		Montevideo Fluvial Bus Station		Point 50.414436
	limites_barrios-63	PUERTO		Polygon
geonames_uy_montevideo-8521751		Tres Cruces Bus Station		Point 50.414436
	limites_barrios-50	TRES CRUCES		Polygon
v_mdg_vias-6834690		GREGORIO MAS DE AYALA		LineString 40.88685
	v_mdg_vias-282610	GUARUYALO		LineString
	limites_barrios-54	BELVEDERE		Polygon
v_mdg_vias-6834691		GREGORIO MAS DE AYALA		LineString 40.88685
	v_mdg_vias-6834689	SANTA LUCIA		LineString
	limites_barrios-54	BELVEDERE		Polygon
v_mdg_vias-6877344		JOSE MAS DE AYALA		LineString 40.88685
	v_mdg_vias-6877345	DIONISIO FERNANDEZ		LineString
	limites_barrios-61	VILLA GARCIA MANGA RURAL		Polygon
v_mdg_vias-6619184		GREGORIO MAS DE AYALA		LineString 40.88685
	v_mdg_vias-282481	EMANCIPACION		LineString
	limites_barrios-54	BELVEDERE		Polygon
v_mdg_vias-5136802		GREGORIO MAS DE AYALA		LineString 40.88685
	v_mdg_vias-5136805	QUEGUAY CHICO		LineString
	limites_barrios-54	BELVEDERE		Polygon
v_mdg_vias-7090394		DR 

	limites_barrios-23	UNION		Polygon
v_mdg_vias-7594589		AV ITALIA		LineString 14.311817
	limites_barrios-14	CARRASCO		Polygon
v_mdg_vias-7594590		AV ITALIA		LineString 14.311817
	limites_barrios-14	CARRASCO		Polygon
v_mdg_vias-7595824		AV ITALIA		LineString 14.311817
	v_mdg_vias-7595942	PAULINA LUISI		LineString
	limites_barrios-11	MALVIN		Polygon
v_mdg_vias-7595831		AV ITALIA		LineString 14.311817
	v_mdg_vias-7595829	ADRIAN TROITIO		LineString
	limites_barrios-11	MALVIN		Polygon
v_mdg_vias-6373500		AV ITALIA		LineString 14.311817
	v_mdg_vias-7093085	BV JOSE BATLLE Y ORDOEZ		LineString
	limites_barrios-23	UNION		Polygon
v_mdg_vias-5164738		AV ITALIA		LineString 14.311817
	v_mdg_vias-5164757	AV DR AMERICO RICALDONI		LineString
	limites_barrios-10	PQUE BATLLE VILLA DOLORES		Polygon
v_mdg_vias-5164808		AV ITALIA		LineString 14.311817
	v_mdg_vias-5164807	AV DR AMERICO RICALDONI		LineString
	limites_barrios-10	PQUE BATLLE VILLA DOLORES		Polygon
v_mdg_vias-5164810		AV ITALIA		LineString 14.31

In [215]:
matchs

[]

In [98]:
results = es.search(index="montevideo", body=
{
    "from" : 0, "size" : 30,
    "query" : {
        "query_string" : {
            "query" : text,
            "analyzer": "calle_analyzer"
        }
    }
})
for result in results['hits']['hits']:
    print(result['_score'],result['_id'],result['_source']['type'],result['_source']['nombre'], result['_source']['geometry']['type'])

(6.5457935, u'v_mdg_vias-287716', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-7084344', u'v_mdg_vias', u'WENCESLAO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-6433252', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-6433297', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.5457935, u'v_mdg_vias-287853', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(6.527852, u'geonames_uy_montevideo-7211405', u'geonames_uy_montevideo', u'Hospital Policial Inspector General Uruguay Genta', u'Point')
(5.572977, u'v_mdg_vias-7362536', u'v_mdg_vias', u'JUAN CRUZ VARELA', u'LineString')
(5.572977, u'v_mdg_vias-7595328', u'v_mdg_vias', u'AV JACOBO VARELA', u'LineString')
(5.572977, u'v_mdg_vias-7595334', u'v_mdg_vias', u'AV JACOBO VARELA', u'LineString')
(5.572977, u'v_mdg_vias-6373779', u'v_mdg_vias', u'AV JACOBO VARELA', u'LineString')
(5.572977, u'v_mdg_vias-295791', u'v_mdg_vias', u'JUAN CRUZ VARELA', u'LineString')
(5.572977, 

In [99]:

for result in results_2:


(0.0, u'v_mdg_vias-287716', u'v_mdg_vias', u'FLORENCIO VARELA', u'LineString')
(0.0, u'v_mdg_vias-287824', u'v_mdg_vias', u'ANZANI', u'LineString')
(0.0, u'limites_barrios-10', u'limites_barrios', u'PQUE BATLLE VILLA DOLORES', u'Polygon')
